In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# 定义参数
# region = 'daqing'
# sample_region = 'DAQING'
# region = 'tianjin'
# sample_region = 'TIANJIN'
region = 'wuhan'
sample_region = 'WUHAN'
# region = 'prd'
# sample_region = 'PRD'
# region = 'tibet'
# sample_region = 'TIBET'

In [4]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [5]:
# 移除无效像元
# .reproject('EPSG:3857',None,30)
# elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
elevation = ee.Image("users/311605001111/hillshade_" + region)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)
def maskSR_reverse(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    image_cloud = img.updateMask(mask.remap([0,1],[1,0]))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image_shadow = img.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True).remap([0,1],[1,0]))
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    image_outlier = img.updateMask(bands.gt(0).rename('outlier'))
    return ee.ImageCollection([image_cloud,image_shadow,image_outlier]).sum()


# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)


# 方案一

In [6]:
# 设置采样个数
number_land = 250
number_water = 3500
# number_land = 250
# number_water = 1500
# 基于图像的样本采集函数
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'point_type':'permanent_water','Image_id':image.get('system:id')}))
    # 陆地
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'point_type':'no_water','Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points]).flatten()
    return sample_points

## 设置采样个数
training = 1500
# training = 4000
# training = 20000
def season(image):
    season_points = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 1,
    })
    water_points = season_points.map(lambda i : i.setMulti({'point_type':'season','Image_id':image.get('system:id')}))
    return water_points

In [7]:
for year in range(1999,2005,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)  
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)           
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
    sample_image = l8.merge(l7).merge(l5).map(water_index).map(maskSR)
    print('影像总数：',sample_image.size().getInfo())
    # 计算交集
    JRC_id = 'JRC/GSW1_3/YearlyHistory/' + str(year)
    Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + str(year)
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
    JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
    Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
    Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
    Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
    Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
    permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
    landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
    seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
    seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
    # 采样
    points_collection = sample_image.map(season).flatten()
    # print(points_collection.size().getInfo())
    dataset_id = region + '_kmean100_season' + str(year) + '_v3'
    assetID = 'users/311605001111/SAMPLE_' + sample_region + '/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': points_collection,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start() 
#     points_collection = sample_image.map(imageSample).flatten()    
#     dataset_id = region + '_kmean100_noseason' + str(year)
#     assetID = 'users/311605001111/SAMPLE_' + sample_region + '/' + dataset_id
#     task = ee.batch.Export.table.toAsset(**{
#         'collection': points_collection,
#         'description': dataset_id,
#         'assetId': assetID
#     })
#     task.start()


1999
影像总数： 37
151
2000
影像总数： 70
765
2001
影像总数： 89
1405
2002
影像总数： 81
1290
2003
影像总数： 79
1671
2004
影像总数： 93
1802


# 方案二

## 永久性水体和非水体

In [ ]:
# 设置采样个数
# number_land = 150
# number_water = 2000
number_land = 250
number_water = 1500
# 基于图像的样本采集函数
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'point_type':'permanent_water','Image_id':image.get('system:id')}))
    # 陆地
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'point_type':'no_water','Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points]).flatten()
    return sample_points

In [ ]:
# ## 永久性水体样本和非水体样本，抽样
# pw_nw = sample_image.map(imageSample).flatten()
# permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))
# no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))
# points_collection = ee.FeatureCollection([permanent_water,no_water]).flatten()
# # 总的样本点数
# # points_collection = sample_image.map(imageSample).flatten()
# dataset_id = region + '_kmean100_noseason_sample' + year
# assetID = 'users/311605001111/' + dataset_id
# task = ee.batch.Export.table.toAsset(**{
#     'collection': points_collection,
#     'description': dataset_id,
#     'assetId': assetID
# })
# task.start()

In [ ]:
for year in range(2016,2021,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)  
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)           
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
    sample_image = l8.merge(l7).merge(l5).map(water_index).map(maskSR)
    print('影像总数：',sample_image.size().getInfo())
    # 计算交集
    JRC_id = 'JRC/GSW1_3/YearlyHistory/' + str(year)
    Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + str(year)
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
    JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
    Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
    Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
    Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
    Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
    permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
    landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
    seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
    seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
    # 采样
    pw_nw = sample_image.map(imageSample).flatten()
    permanent_water = pw_nw.filter(ee.Filter.eq('waterclass',1)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))
    no_water = pw_nw.filter(ee.Filter.eq('waterclass',0)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.2))
    points_collection = ee.FeatureCollection([permanent_water,no_water]).flatten()
    dataset_id = region + '_kmean100_noseason_sample' + str(year)
    assetID = 'users/311605001111/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': points_collection,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

## 季节性样本

In [ ]:
training = 1500
def filter_image(image):
    samples = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(samples.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1)).size()
    q0 = q.filter(ee.Filter.eq('cluster',0)).size()
    return image.setMulti({'wc':q1,'nwc':q0})
def season(image):
    samples = image.updateMask(seasonWaterExtent_min).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': training,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    season_points = samples.map(lambda i : i.setMulti({'point_type':'season_water','Image_id':image.get('system:id')}))
    label = 'waterclass'
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    clusterer = ee.Clusterer.wekaKMeans(2).train(cluster_samples,bands)
    q = ee.FeatureCollection(season_points.cluster(clusterer))
    q1 = q.filter(ee.Filter.eq('cluster',1))
    q0 = q.filter(ee.Filter.eq('cluster',0))
    b1 = ee.FeatureCollection(q1).filter(ee.Filter.lt('AWEI',0)).size()
    b2 = ee.FeatureCollection(q1).filter(ee.Filter.gt('AWEI',0)).size()
    c1 = ee.FeatureCollection(q0).filter(ee.Filter.lt('AWEI',0)).size()
    c2 = ee.FeatureCollection(q0).filter(ee.Filter.gt('AWEI',0)).size()
    qq1 = ee.Algorithms.If(ee.Number(b1).lt(ee.Number(b2)),q1.map(lambda i:i.set({'waterclass':1})),q1.map(lambda i:i.set({'waterclass':0})))
    qq0 = ee.Algorithms.If(ee.Number(c1).lt(ee.Number(c2)),q0.map(lambda i:i.set({'waterclass':1})),q0.map(lambda i:i.set({'waterclass':0})))
    cluster_result = ee.FeatureCollection([qq1,qq0]).flatten()
    return cluster_result



In [ ]:
for year in range(2017,2021,1):
    print(year)
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)  
    l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)           
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
    sample_image = l8.merge(l7).merge(l5).map(water_index).map(maskSR)
    print('影像总数：',sample_image.size().getInfo())
    # 计算交集
    JRC_id = 'JRC/GSW1_3/YearlyHistory/' + str(year)
    Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + str(year)
    basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
    JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
    JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
    Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
    Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
    Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
    Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
    permanentWaterExtent = ee.ImageCollection([JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(2)
    landExtent = ee.ImageCollection([JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(2)
    seasonWaterExtent_min = ee.ImageCollection([JRC_waterclass.eq(1),Maryland_waterclass.eq(1)]).sum().eq(2)
    seasonWaterExtent_max = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')
    # 采样
    cluster_samples = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_noseason_sample' + str(year))
    points_collection = sample_image.map(filter_image).filter(ee.Filter.gt('wc',0)).filter(ee.Filter.gt('nwc',0)).map(season).flatten()
    dataset_id = region + '_kmean100_season_sample' + str(year)
    assetID = 'users/311605001111/' + dataset_id
    task = ee.batch.Export.table.toAsset(**{
        'collection': points_collection,
        'description': dataset_id,
        'assetId': assetID
    })
    task.start()

In [ ]:
# # points_collection = sample_image.map(imageSample).flatten()
# points_collection = sample_image.map(filter_image).filter(ee.Filter.gt('wc',0)).filter(ee.Filter.gt('nwc',0)).map(season).flatten()

# dataset_id = region + '_kmean100_season_samplev2' + year
# assetID = 'users/311605001111/' + dataset_id
# task = ee.batch.Export.table.toAsset(**{
#     'collection': points_collection,
#     'description': dataset_id,
#     'assetId': assetID
# })
# task.start()

# 分析

In [ ]:
## 异常值去除
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
#### cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
#### k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [ ]:
region = 'tianjin'
totalSamples = ee.FeatureCollection([])
for year in range(2016,2021,1):
    print(year)
    ## 永久性水体样本和非水体样本，抽样
    pw_nw = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_noseason_sample' + str(year))
    ## 季节性样本，抽样
    sw = ee.FeatureCollection('users/311605001111/' + region + '_kmean100_season_sample' + str(year)).randomColumn('random',1,'uniform').filter(ee.Filter.lt('random',0.4))
    ## 合并
    total_Samples = ee.FeatureCollection([pw_nw,sw]).flatten()
    ## 异常值去除
    filtered_samples = k_mean(total_Samples)
    print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())
    print('no-water')
    print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
    print(filtered_samples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
    print('water')
    print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
    print(filtered_samples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
    totalSamples = totalSamples.merge(filtered_samples)
    print("总样本： ",totalSamples.size().getInfo())

In [ ]:
print('no-water')
print(totalSamples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(totalSamples.filter(ee.Filter.eq('waterclass',0)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())
print('water')
print(totalSamples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.lt('AWEI',0)).size().getInfo())
print(totalSamples.filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.gt('AWEI',0)).size().getInfo())